In [21]:
import pandas as pd
import csv
import yfinance as yf

#Insider Data Amazon
insider_df = pd.read_csv('C:/Users/trist/code/Data/Finviz_insider/Data/AMZN.csv')
# Stock Data Amazon
ticker_symbol = 'AMZN'
start_date = '2023-01-01'
end_date = '2023-11-30'
amazon_df = yf.download(ticker_symbol, start=start_date, end=end_date)

# insider_df: name='Date', dtype= 'object'
# amazon_df: name='Date', dtype='datetime64[ns]' 


[*********************100%%**********************]  1 of 1 completed


In [22]:
#Inner-Join
insider_df['Date'] = pd.to_datetime(insider_df['Date'])
result_df = pd.merge(insider_df, amazon_df, left_on='Date', right_index=True, how='inner')

x = len(result_df)
#print(result_df)
columns_list = result_df.columns.tolist()
# # index_vals = amazon_df.index
print(x)


#ein paar Einträge scheinen zu fehlen... vielleicht weil es keine Daten über das Wochenende gibt? muss das Überprüfen


94


In [5]:
import pandas as pd

# Example data (replace this with your actual DataFrames)
insider_trades = pd.DataFrame({
    'timestamp': ['2023-01-01', '2023-02-15', '2023-03-10'],
    'insider_activity': ['Buy', 'Sell', 'Buy'],
    # other insider trade columns
})

stock_prices = pd.DataFrame({
    'timestamp': ['2023-01-01', '2023-01-02', '2023-02-15', '2023-02-16', '2023-03-10'],
    'stock_price': [100, 105, 110, 108, 115],
    # other stock price columns
})

# Convert timestamp columns to datetime objects
insider_trades['timestamp'] = pd.to_datetime(insider_trades['timestamp'])
stock_prices['timestamp'] = pd.to_datetime(stock_prices['timestamp'])

# Specify the time window for stock prices around each insider trade
time_window_days = 3

# Perform the join based on the time window
enriched_data = pd.merge_asof(
    insider_trades,
    stock_prices,
    on='timestamp',
    direction='nearest',  # Choose the nearest timestamp within the time window
    tolerance=pd.Timedelta(days=time_window_days)
)

# Display the resulting DataFrame
print(enriched_data)

   timestamp insider_activity  stock_price
0 2023-01-01              Buy          100
1 2023-02-15             Sell          110
2 2023-03-10              Buy          115
